<a href="https://colab.research.google.com/github/biini/ai06project3/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install category_encoders

     |████████████████████████████████| 80 kB 6.2 MB/s 


In [2]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from category_encoders import OneHotEncoder
import joblib
import pickle

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Get data
path = '/content/drive/MyDrive/Colab Notebooks/Section3/book_data.db'
conn = sqlite3.connect(path)
cur = conn.cursor()

In [4]:
data = pd.read_sql('SELECT * FROM Book', conn)
data

,ranking,item_id,sales_point,rating,genre
0,1,280840739,"305,050",9.8,만화
1,2,280598932,"96,220",10.0,만화
2,3,278770576,"200,510",8.9,소설/시/희곡
3,4,280637907,"49,310",7.7,만화
4,5,279711074,"90,100",7.8,경제경영
...,...,...,...,...,...
193,194,279815126,"8,780",9.8,자기계발
194,195,277669618,"8,520",10.0,수험서/자격증
195,196,280421937,"5,080",0.0,경제경영
196,197,7165205,"104,111",9.8,어린이


In [5]:
# sales_point: Remove ',' and turn data into integer
data['sales_point'] = pd.to_numeric(data['sales_point'].str.replace(',',''))

data

,ranking,item_id,sales_point,rating,genre
0,1,280840739,305050,9.8,만화
1,2,280598932,96220,10.0,만화
2,3,278770576,200510,8.9,소설/시/희곡
3,4,280637907,49310,7.7,만화
4,5,279711074,90100,7.8,경제경영
...,...,...,...,...,...
193,194,279815126,8780,9.8,자기계발
194,195,277669618,8520,10.0,수험서/자격증
195,196,280421937,5080,0.0,경제경영
196,197,7165205,104111,9.8,어린이


In [6]:
# Remove 'item_id'
data.drop('item_id', axis=1, inplace=True)
data

,ranking,sales_point,rating,genre
0,1,305050,9.8,만화
1,2,96220,10.0,만화
2,3,200510,8.9,소설/시/희곡
3,4,49310,7.7,만화
4,5,90100,7.8,경제경영
...,...,...,...,...
193,194,8780,9.8,자기계발
194,195,8520,10.0,수험서/자격증
195,196,5080,0.0,경제경영
196,197,104111,9.8,어린이


In [7]:
# Encoding
encoder = OneHotEncoder(use_cat_names=True)
data_encoded = encoder.fit_transform(data)

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [8]:
data_encoded

,ranking,sales_point,rating,genre_만화,genre_소설/시/희곡,genre_경제경영,genre_어린이,genre_건강/취미/레저,genre_에세이,genre_과학,genre_유아,genre_예술/대중문화,genre_인문학,genre_수험서/자격증,genre_좋은부모,genre_자기계발,genre_외국어,genre_사회과학,genre_청소년,genre_가정/요리/뷰티,genre_컴퓨터/모바일
0,1,305050,9.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,96220,10.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,200510,8.9,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,49310,7.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,90100,7.8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,194,8780,9.8,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
194,195,8520,10.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
195,196,5080,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
196,197,104111,9.8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Set target and features
target = data_encoded['sales_point']
features = data_encoded.drop('sales_point', axis=1)

# Split data into train and test data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state = 3)

In [10]:
# length of train and test data
len(X_train), len(X_test)

(158, 40)

In [11]:
len(X_train.columns)

20

In [12]:
len(X_test.columns)

20

In [13]:
# Set Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

# Inference the model to test data
y_pred = model.predict(X_test)


In [14]:
# Put data into model and predict the sales point
model.predict([[2,9.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

array([82868.51092994])

In [15]:
X_test.columns

Index(['ranking', 'rating', 'genre_만화', 'genre_소설/시/희곡', 'genre_경제경영',
       'genre_어린이', 'genre_건강/취미/레저', 'genre_에세이', 'genre_과학', 'genre_유아',
       'genre_예술/대중문화', 'genre_인문학', 'genre_수험서/자격증', 'genre_좋은부모',
       'genre_자기계발', 'genre_외국어', 'genre_사회과학', 'genre_청소년', 'genre_가정/요리/뷰티',
       'genre_컴퓨터/모바일'],
      dtype='object')

In [16]:
# Save the model
with open('point_predict.pickle','wb') as file:
  pickle.dump(model, file)

In [17]:
# Load the model
model1 = pickle.load(open('point_predict.pickle','rb'))

In [19]:
# Put data into model and predict sales point
prediction = model1.predict([[2,9.3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
prediction

array([82868.51092994])